In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [3]:
from lib.training_data import *

ModuleNotFoundError: No module named 'lib'

In [ ]:
file_path = "../data/adult_earning/"
!ls $file_path

In [ ]:
hfile = file_path + "adult.names"
header = []
with open(hfile, "r") as infile:
    for line in infile:
        if line.startswith("|") or line.startswith(">") or line.startswith("\n"):
            continue
        else:
            header.append(line.split(":")[0])
header.append("class")
print(header)

In [ ]:
dfile = file_path + "adult.data"
df = pd.read_csv(dfile, names=header)
df.head()

In [ ]:
df.info()

In [ ]:
for h in header:
    n = len(df[h].value_counts())
    if n < 20:
        print("###############################")
        print(f"{h}: {n}")
        print(df[h].value_counts())

In [ ]:
# output encodings
df["gt50k"] = df["class"].apply(lambda x: 1 if ">" in x else 0)
df["lte50k"] = df["class"].apply(lambda x: 1 if "<=" in x else 0)
# input encodings
df["sex-val"] = df["sex"].apply(lambda x: 1 if "M" in x else 0)

In [ ]:
df.head()

In [ ]:
dft = df[['age','fnlwgt','sex-val','education-num', "capital-gain", "capital-loss", "hours-per-week"]].copy()
y = df["class"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(dft, y)

In [ ]:
dec_tree=DecisionTreeClassifier().fit(X_train, y_train)
dec_tree.score(X_test, y_test)

In [ ]:
y_preds = dec_tree.predict(X_test)
print(classification_report(y_test, y_preds))

In [ ]:
#Normalizing the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
test_dec_tree=DecisionTreeClassifier().fit(X_train, y_train)

In [ ]:
test_dec_tree.score(X_test, y_test)

In [ ]:
y_preds = test_dec_tree.predict(X_test)
print(classification_report(y_test, y_preds))

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
dy = df[["gt50k", "lte50k"]]
X_train, X_test, y_train, y_test = train_test_split(dft, dy)
#Normalizing the data
from sklearn.preprocessing import StandardScaler
nnsc = StandardScaler()
X_train = nnsc.fit_transform(X_train)
X_test = nnsc.transform(X_test)

In [ ]:
# Neural network
model = Sequential()
model.add(Dense(20, input_dim=7, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train,validation_data = (X_test,y_test), epochs=100, batch_size=64)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
#Converting predictions to label
pred = list()
for i in range(len(y_pred)):
    pred.append(np.argmax(y_pred[i]))

In [ ]:
test = list(y_test.lte50k)

In [ ]:
from sklearn.metrics import accuracy_score
a = accuracy_score(pred,test)
print('Accuracy is:', a*100)

In [ ]:
print(classification_report(test, pred))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

In [ ]:
from joblib import dump, load
dump(dec_tree, "decision_tree.pkl")
dump(model, "neural_net.pkl")
# dump(sc, "decision_tree_scaler.pkl")
dump(nnsc, "neural_net_scaler.pkl")